In [1]:
from keras.models import Sequential  
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from keras import layers, models, optimizers, losses
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import os 
import glob

In [2]:
# CARREGAMENTO DOS DATASETS

''' 
    Carregando a pasta de imagens (Dataset dos ovos)
    * A função glob encontra todos os nomes de caminhos que correspondem a um padrão 
      especificado de acordo com as regras usadas pelo shell Unix

    * Para cada um dos arquivos encontrados é feita a leitura
      do arquivo .jpg (imagem), as imagens lidas são armazenadas
      na lista 'data'
'''

#img_dir = "C:/Users/marci/Desktop/Projeto mestrado/CNN Egg application/Egg Dataset"
img_dir = "C:/Users/marci/Desktop/Arquivos/PRJ_mestrado/CNN Egg application/Egg Dataset"
data_path = os.path.join(img_dir,'*g') 

folder = glob.glob(data_path) 
data = [] 
for files in folder: 
    img = cv2.imread(files) 
    data.append(img) 

'''   
    Carregando e tratando o arquivo CSV
'''
eggClass = pd.read_csv("RealData.csv")
cList = eggClass.to_numpy()

print(len(data))
print(cList.shape)

5100
(5100, 1)


In [5]:
# Dataset Completo
# Comentar seção caso não seja necessário

ind = np.arange(len(data))
train, test = train_test_split(ind, test_size=0.2, random_state=42)

#
dataTrain = []
dataTest = []
classTrain = []
classTest = []

#
for i in range(len(train)):

    dataTrain.append(data[train[i]])
    classTrain.append(cList[train[i]])

for j in range(len(test)):

    dataTest.append(data[test[j]])
    classTest.append(cList[test[j]])

#
dataTrain = np.array(dataTrain)
dataTest = np.array(dataTest)
classTrain = np.array(classTrain)
classTest = np.array(classTest)

print('img train', len(dataTrain))
print('img test', len(dataTest))
print('class train', len(classTrain))
print('class test', len(classTest))


img train 4080
img test 1020
class train 4080
class test 1020


In [6]:
del data
del cList

In [7]:
'''
    Alterando o tipo dos dados para float32 a fim de aplicar 
    a normalização futuramente
'''

dataTrain = dataTrain.astype('float32')
dataTest = dataTest.astype('float32')

'''
    Realizando a normalização (min/max normalization) a fim de que os valores dos pixels estejam
    entre 0 e 1, tornando o processamento mais eficiente
    obs: 255 é o valor máximo do pixel
'''

dataTrain /= 255
dataTest /= 255

In [3]:
# SHALLOW NETWORK

classifier = Sequential()

classifier.add(Conv2D(32, (3,3), input_shape = (512, 512, 3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Conv2D(32, (3,3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [4]:
# ALEXNET

model = Sequential()

model.add(Conv2D(96, (11, 11), strides=4, activation='relu', input_shape = (512, 512, 3), padding='valid'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Conv2D(256, (5, 5), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))
    
model.add(Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))
    
model.add(Conv2D(256, (3, 3), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [10]:
# MOBILE NET PRÉ-TREINADO

base_model = tf.keras.applications.MobileNetV2(input_shape=(512, 512, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False  # Congelar as camadas do modelo base

model2 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model2.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss=losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

C:\Users\marci\AppData\Local\Temp\ipykernel_7028\2331232953.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(512, 512, 3),


In [11]:
# TREINO ALEXNET

board = TensorBoard(log_dir='./logsAlex')
model.fit(dataTrain, classTrain, batch_size = 32, epochs = 10, validation_data = (dataTest, classTest), callbacks=[board])

Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 957s 7s/step - accuracy: 0.6908 - loss: 0.7963 - val_accuracy: 0.7343 - val_loss: 0.5816
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 929s 7s/step - accuracy: 0.7448 - loss: 0.5716 - val_accuracy: 0.7343 - val_loss: 0.5790
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 939s 7s/step - accuracy: 0.7276 - loss: 0.5902 - val_accuracy: 0.7343 - val_loss: 0.5870
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 926s 7s/step - accuracy: 0.7225 - loss: 0.5968 - val_accuracy: 0.7343 - val_loss: 0.5808
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 929s 7s/step - accuracy: 0.7433 - loss: 0.5712 - val_accuracy: 0.7343 - val_loss: 0.5818
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 937s 7s/step - accuracy: 0.7255 - loss: 0.5879 - val_accuracy: 0.7343 - val_loss: 0.5789
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 930s 7s/step - accuracy: 0.7273 - loss: 0.5890 - val_accuracy: 0.7343 - val_loss: 0.5789
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 931s 7s/step - accuracy: 0.7265 - loss: 0.5872 - val_accu

In [13]:
board2 = TensorBoard(log_dir='./logsShallow')
classifier.fit(dataTrain, classTrain, batch_size = 32, epochs = 10, validation_data = (dataTest, classTest), callbacks=[board2])

Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 614s 5s/step - accuracy: 0.6459 - loss: 20.1427 - val_accuracy: 0.7343 - val_loss: 1.9548
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 587s 5s/step - accuracy: 0.7291 - loss: 0.6325 - val_accuracy: 0.7343 - val_loss: 4.0500
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 598s 5s/step - accuracy: 0.7255 - loss: 0.6035 - val_accuracy: 0.7343 - val_loss: 4.1849
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 593s 5s/step - accuracy: 0.7276 - loss: 0.5897 - val_accuracy: 0.7343 - val_loss: 1.4946
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 594s 5s/step - accuracy: 0.7171 - loss: 0.5951 - val_accuracy: 0.7343 - val_loss: 0.5794
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 593s 5s/step - accuracy: 0.7258 - loss: 0.5868 - val_accuracy: 0.7343 - val_loss: 0.5790
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 594s 5s/step - accuracy: 0.7355 - loss: 0.5786 - val_accuracy: 0.7343 - val_loss: 0.5789
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 618s 5s/step - accuracy: 0.7343 - loss: 0.5791 - val_acc

In [15]:
board3 = TensorBoard(log_dir='./logsMobile')
model2.fit(dataTrain, classTrain, batch_size = 32, epochs = 10, validation_data = (dataTest, classTest), callbacks=[board3])


Epoch 1/10


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\backend\tensorflow\nn.py:559: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


128/128 ━━━━━━━━━━━━━━━━━━━━ 396s 3s/step - accuracy: 0.2842 - loss: 0.0000e+00 - val_accuracy: 0.2657 - val_loss: 0.0000e+00
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 377s 3s/step - accuracy: 0.2734 - loss: 0.0000e+00 - val_accuracy: 0.2657 - val_loss: 0.0000e+00
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 376s 3s/step - accuracy: 0.2716 - loss: 0.0000e+00 - val_accuracy: 0.2657 - val_loss: 0.0000e+00
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 375s 3s/step - accuracy: 0.2690 - loss: 0.0000e+00 - val_accuracy: 0.2657 - val_loss: 0.0000e+00
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 373s 3s/step - accuracy: 0.2718 - loss: 0.0000e+00 - val_accuracy: 0.2657 - val_loss: 0.0000e+00
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 377s 3s/step - accuracy: 0.2754 - loss: 0.0000e+00 - val_accuracy: 0.2657 - val_loss: 0.0000e+00
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 375s 3s/step - accuracy: 0.2679 - loss: 0.0000e+00 - val_accuracy: 0.2657 - val_loss: 0.0000e+00
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 382s 3s/step

In [16]:
#model.save_weights('AlexNet10ep.weights.h5')
#model2.save_weights('mobileNet10ep.weights.h5')
#classifier.save_weights('shallownet.weights.h5')

In [5]:
# Carregando pesos

model.load_weights('AlexNet10ep.weights.h5')
classifier.load_weights('shallownet.weights.h5')

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [6]:
npData = np.array(data)
alexPredict = model.predict(npData)
shallowPredict = classifier.predict(npData)


160/160 ━━━━━━━━━━━━━━━━━━━━ 308s 2s/step
160/160 ━━━━━━━━━━━━━━━━━━━━ 106s 660ms/step


In [43]:
npAlex = np.array(alexPredict)
npShallow = np.array(shallowPredict)

print(npAlex)
print('-----------------')
print(npShallow)

[[0.74711275]
 [0.747967  ]
 [0.7496995 ]
 ...
 [0.7487219 ]
 [0.7488048 ]
 [0.7477565 ]]
-----------------
[[0.72962284]
 [1.        ]
 [1.        ]
 ...
 [0.72962284]
 [0.72962284]
 [1.        ]]


In [58]:
AlexClass = (npAlex >= 0.7343).astype(int)
ShallowClass = (npShallow >= 0.7343).astype(int)

In [59]:
AlexClass = np.ravel(AlexClass)
ShallowClass = np.ravel(ShallowClass)
print(AlexClass)
print(ShallowClass)

[1 1 1 ... 1 1 1]
[0 1 1 ... 0 0 1]


In [60]:
count = 0
count2 = 0
for i in range(len(AlexClass)):
    if AlexClass[i] == 1:
        count += 1
    
    if ShallowClass[i] == 1:
        count2 += 1

print('alex',count)
print('shallow',count2)

alex 5100
shallow 3559
